<center>
    <font size="5"> Zaawansowane Metody Uczenia Maszynowego i Głębokiegio<br/>
        <small><em>Studia stacjonarne II stopnia 2025/2026</em><br/>Kierunek: Informatyka<br>Specjalność: Systemy inteligentne i rozszerzona rzeczywistość</small>
    </font>
</center>
<br>



# Laboratorium nr 11: RNN - Zadanie

Głównym zadaniem jest stworzenie i wytrenowanie głębokiej rekurencyjnej sieci neuronowej zdolnej poprawnie rozwiązać jedno z zagadnień rozpoznawania mowy a mianowicie zagadnienie identyfikacji rozmówcy.

Dane uczące zawierają nagrania audio pochodzące z rozmowy pomiędzy osobą A i B (plik `dataset.zip` dołączony do zadania).

W trakcie realizacji zadania należy wykonań następujące podzadania:
1. Podzielić próbki na dane uczące i testowe.
2. Zdefiniować __dwa__ modele głebokiej rekurencyjnej sieci neuronowej według własnego pomysłu. W pierwszym modelu dzwięk reprezentowany powinien być za pomocą szeregu czasowego, a w drugim za pomocą spektogramu MFCC (warto wykorzystać pakiet [librosa](https://librosa.github.io/librosa/generated/librosa.feature.mfcc.html). Oba modele powinny rozwiązywać problem klasyfikacji, czy dana próbka pochodzi od osoby A czy B.
3. Wytrenować zdefiniowane sieci na danych uczących.
4. Ocenić skuteczność i porównać działanie modeli na danych testowych.
5. Najlepsze ze stworzonych modeli zapisać do pliku i przesłać na elf'a razem z notatnikiem.

__Polecam też rozszerzyć zbiór danych o nagrania własnego głosu.__

## Import bibliotek

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import librosa
print('Numpy version:', np.__version__)
print('Tensorflow version:', tf.__version__)
print('Keras version:', tf.keras.__version__)

import os
import zipfile
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

Numpy version: 2.2.6
Tensorflow version: 2.20.0
Keras version: 3.11.3


## Przygotowanie danych

In [21]:
DATA_DIR = "/mnt/d/GitProj/Jupyter/RNN2/dataset"


def load_raw_audio(path, sr=16000, max_len=16000):
    y, _ = librosa.load(path, sr=sr)
    if len(y) > max_len:
        y = y[:max_len]
    else:
        y = np.pad(y, (0, max_len - len(y)))
    return y

def load_mfcc(path, sr=16000, n_mfcc=13, max_len=100):
    y, _ = librosa.load(path, sr=sr)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mfcc = mfcc.T
    if mfcc.shape[0] > max_len:
        mfcc = mfcc[:max_len, :]
    else:
        pad = max_len - mfcc.shape[0]
        mfcc = np.pad(mfcc, ((0, pad), (0, 0)))
    return mfcc


def build_dataset(feature_fn):
    X, y = [], []

    for label, speaker in enumerate(["A", "B"]):
        folder = os.path.join(DATA_DIR, speaker)

        for file in os.listdir(folder):
            if file.endswith(".wav"):
                path = os.path.join(folder, file)
                X.append(feature_fn(path))
                y.append(label)

    return np.array(X), np.array(y)


X_raw, y_raw = build_dataset(load_raw_audio)
X_raw = X_raw[..., np.newaxis]   # (N, T, 1)

Xr_train, Xr_test, yr_train, yr_test = train_test_split(
    X_raw, y_raw, test_size=0.2, random_state=42
)

X_mfcc, y_mfcc = build_dataset(load_mfcc)

Xm_train, Xm_test, ym_train, ym_test = train_test_split(
    X_mfcc, y_mfcc, test_size=0.2, random_state=42
)

## Definicja i trenowanie modeli

In [22]:
model_raw = Sequential([
    LSTM(64, return_sequences=True, input_shape=Xr_train.shape[1:]),
    LSTM(64),
    Dense(1, activation="sigmoid")
])

model_raw.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model_raw.summary()


model_mfcc = Sequential([
    LSTM(128, return_sequences=True, input_shape=Xm_train.shape[1:]),
    LSTM(128),
    Dense(1, activation="sigmoid")
])

model_mfcc.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model_mfcc.summary()

print("Trening RAW AUDIO")
model_raw.fit(Xr_train, yr_train, epochs=10, batch_size=16, validation_split=0.1)

print("Trening MFCC")
model_mfcc.fit(Xm_train, ym_train, epochs=10, batch_size=16, validation_split=0.1)

E0000 00:00:1766958128.435042    1337 cuda_executor.cc:1309] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1766958128.448535    1337 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/user/anaconda3/envs/srod/lib/python3.13/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 16000, 64)      │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 49,985 (195.25 KB)

 Trainable params: 49,985 (195.25 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 100, 128)       │        72,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 204,417 (798.50 KB)

 Trainable params: 204,417 (798.50 KB)

 Non-trainable params: 0 (0.00 B)

Trening RAW AUDIO
Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 40s 13s/step - accuracy: 0.4595 - loss: 0.6924 - val_accuracy: 0.4000 - val_loss: 0.6985
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 38s 13s/step - accuracy: 0.5946 - loss: 0.6872 - val_accuracy: 0.4000 - val_loss: 0.7100
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 38s 13s/step - accuracy: 0.5946 - loss: 0.6770 - val_accuracy: 0.4000 - val_loss: 0.7391
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 37s 13s/step - accuracy: 0.5946 - loss: 0.6746 - val_accuracy: 0.4000 - val_loss: 0.7535
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 38s 13s/step - accuracy: 0.5946 - loss: 0.6716 - val_accuracy: 0.4000 - val_loss: 0.7830
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 38s 11s/step - accuracy: 0.5946 - loss: 0.6860 - val_accuracy: 0.4000 - val_loss: 0.8075
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 40s 12s/step - accuracy: 0.5946 - loss: 0.6849 - val_accuracy: 0.4000 - val_loss: 0.7640
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 40s 12s/step - accuracy: 0.5946 - loss: 0.6745 - val_accuracy: 0.4000 -

## Ocena skuteczności

In [23]:
print("RAW AUDIO:")
pred_raw = (model_raw.predict(Xr_test) > 0.5).astype(int)
print(classification_report(yr_test, pred_raw))

print("MFCC:")
pred_mfcc = (model_mfcc.predict(Xm_test) > 0.5).astype(int)
print(classification_report(ym_test, pred_mfcc))

model_mfcc.save("speaker_id_mfcc_model.keras")

RAW AUDIO:
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
              precision    recall  f1-score   support

           0       0.55      1.00      0.71         6
           1       0.00      0.00      0.00         5

    accuracy                           0.55        11
   macro avg       0.27      0.50      0.35        11
weighted avg       0.30      0.55      0.39        11

MFCC:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step


/home/user/anaconda3/envs/srod/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/user/anaconda3/envs/srod/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/user/anaconda3/envs/srod/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()}

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00         5

    accuracy                           1.00        11
   macro avg       1.00      1.00      1.00        11
weighted avg       1.00      1.00      1.00        11

